In [38]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta
import math
import re

# Reading Source Data 'ADT_Dummy Data' and 'ADT_ER_IP_04012021to07062021'

Auth_data = pd.read_excel(r'C:\Users\ssing296\Desktop\Canopy\Data Files\Auth Data\Auth.xlsx')

df = pd.read_excel(r'C:\Users\ssing296\Desktop\Canopy\CanopyConnect High Risk Codes.xlsx')

df = df.drop_duplicates(keep='first')
#print(Auth_data["DX1"])
Auth_data["DX1"] = Auth_data["DX1"].replace(",",";").apply(str)

Auth_data["DX1"].append(Auth_data["DX2"]).reset_index(drop=True)
Auth_data["DX_combined"] = Auth_data["DX1"].apply(str) +";" + Auth_data["DX2"].apply(str)+";" + Auth_data["DX3"].apply(str)+";" + Auth_data["DX4"].apply(str)+";" + Auth_data["DX5"].apply(str)+";" + Auth_data["DX6"].apply(str)+";" + Auth_data["DX7"].apply(str)+";" + Auth_data["DX8"].apply(str)+";" + Auth_data["DX9"].apply(str)
#print(Auth_data["DX_combined"])
high_cost_codes = pd.read_excel(r'C:\Users\ssing296\Desktop\Canopy\High Cost Chronic Condition Codes.xlsx')

# filling 'blank_code' in place of null values

Auth_data['DX_combined'].fillna('blank_code', inplace=True)

# making list of diagnosis, high-risk and high-cost code

diagnosis_lst = Auth_data['DX_combined'].tolist()
#print(diagnosis_lst)
canopy_high_risk_codes = df['code'].tolist()

canopy_high_risk_prim_d = df['Primary Disease'].tolist()

high_cost_chronic_codes = high_cost_codes['code'].tolist()

high_cost_chronic_subcategory = high_cost_codes['Subcategory'].tolist()

text_code = ['abdominal pain','chest pain','back pain','breath','SOB','short of breath','fall','pregnancy','covid']

for i in range(len(canopy_high_risk_codes)):
    if(isinstance(canopy_high_risk_codes[i],int)):
        canopy_high_risk_codes[i]
    else:
        canopy_high_risk_codes[i] = canopy_high_risk_codes[i].lower()

for i in range(len(high_cost_chronic_codes)):
    if(isinstance(high_cost_chronic_codes[i],int)):
        high_cost_chronic_codes[i]
    else:
        high_cost_chronic_codes[i] = high_cost_chronic_codes[i].lower()
        

canopy_high_risk_codes = canopy_high_risk_codes + text_code
high_cost_chronic_codes = high_cost_chronic_codes + text_code


def high_risk_cost_flag (diagnosis_lst,canopy_high_risk_codes):
    b = []
    prim_d = []
    codes = []
    for i in range(0,len(diagnosis_lst)):
        a = diagnosis_lst[i]
        #print(a)
        split1 =  a.split(";")
        split = [item for item in split1 if not(item == 'nan')]
        #print(split)
        clean_lst = []
        
        for i in range(0,len(split)):
            a = split[i].strip()
            a = a.replace(".",'')
            a = a.lower()
            
            if(re.findall(r"abd*",a)):
                a = "abdominal pain"
            elif(re.findall(r"chest*",a)):
                a = "chest pain"
            elif(re.findall(r"back*",a)):
                a = "back pain"
            elif(re.findall(r"breath*",a) or re.findall(r"sob*",a)):
                a = "short of breath"
            elif(re.findall(r"fall*",a)):
                a = "fall"
            elif(re.findall(r"pregnancy*",a)):
                a = "pregnancy"
            elif(re.findall(r"covid*",a)):
                a = "covid"
            else:
                a
            clean_lst.append(a)

        if set(clean_lst).intersection(canopy_high_risk_codes):
            res = "Y"
            b.append(res)
        else:
            res = "N"
            b.append(res)
        
    return b

def high_risk_cost_desc (diagnosis_lst,canopy_high_risk_codes,col_name):
    b = []
    prim_d = []
    codes = []
    
    for i in range(0,len(diagnosis_lst)):
        a = diagnosis_lst[i]
        #split =  a.split(";")
        split1 =  a.split(";")
        split = [item for item in split1 if not(item == 'nan')]

        #print(split)
        clean_lst = []
        for i in range(0,len(split)):
            a = split[i].strip()
            a = a.replace(".",'')
            a = a.lower()
            if(re.findall(r"abd*",a)):
                a = "abdominal pain"
            elif(re.findall(r"chest*",a)):
                a = "chest pain"
            elif(re.findall(r"back*",a)):
                a = "back pain"
            elif(re.findall(r"breath*",a) or re.findall(r"sob*",a)):
                a = "short of breath"
            elif(re.findall(r"fall*",a)):
                a = "fall"
            elif(re.findall(r"pregnancy*",a)):
                a = "pregnancy"
            elif(re.findall(r"covid*",a)):
                a = "covid"
            else:
                a
            clean_lst.append(a)
        
        if set(clean_lst).intersection(canopy_high_risk_codes):
            prim_d = set(clean_lst).intersection(canopy_high_risk_codes)
            prim_d = list(prim_d)
            for i in range(len(prim_d)):
                if(isinstance(prim_d[i],int)):
                    prim_d[i]
                else:
                    prim_d[i] = prim_d[i].upper()
            lsttostr_prim = ','.join([str(elem) for elem in prim_d])
            
            if(col_name == "Primary Disease"):
                res = df[col_name][df['code'].isin(prim_d)].tolist()
            else:
                res = high_cost_codes[col_name][high_cost_codes['code'].isin(prim_d)].tolist()
                
            lst_set = set(res)
            listToStr = ','.join([str(elem) for elem in lst_set])
            if(len(res) == 0):
                b.append(lsttostr_prim)
            else: 
                b.append(listToStr)
                
        else:
            res1 = col_name
            res1  =  "Not a High Risk" if(col_name == "Primary Disease") else "Not a High Cost"
            b.append(res1)
    return b


Auth_data["High Risk Flag"] = high_risk_cost_flag(diagnosis_lst,canopy_high_risk_codes)

Auth_data["High Risk desc"] = high_risk_cost_desc(diagnosis_lst,canopy_high_risk_codes,"Primary Disease")

Auth_data["High Cost Flag"] = high_risk_cost_flag(diagnosis_lst,high_cost_chronic_codes)

Auth_data["High Cost desc"] = high_risk_cost_desc(diagnosis_lst,high_cost_chronic_codes,"Subcategory")

Auth_data.to_excel(r'C:\Users\ssing296\Desktop\Canopy\Data Files\Auth Data\Auth_Python_data_set.xlsx', index = False)